In [6]:
class Regressor:
    def __init__(self):
        self.coefficients = None
    
    def train(self, X, y):
        """
        Antrenează modelul de regresie liniară pe datele de intrare X și valorile target y.
        """
        # Adăugăm o coloană de 1 la intrările X pentru a reprezenta intercept-ul
        X_with_intercept = [[1] + row for row in X]

        # Calculăm coeficienții folosind metoda minimelor pătratelor
        X_transpose = self.transpose(X_with_intercept)
        XTX_inverse = self.inverse(self.matrix_multiply(X_transpose, X_with_intercept))
        self.coefficients = self.matrix_multiply(self.matrix_multiply(XTX_inverse, X_transpose), y)
    
    def predict(self, X):
        """
        Efectuează predicții pe datele de intrare X.
        """
        # Adăugăm o coloană de 1 la intrările X pentru a reprezenta intercept-ul
        X_with_intercept = [[1] + row for row in X]

        # Efectuăm predicții folosind coeficienții modelului
        return [self.dot(row, self.coefficients) for row in X_with_intercept]
    
    def transpose(self, A):
        """
        Transpune o matrice.
        """
        return [[A[j][i] for j in range(len(A))] for i in range(len(A[0]))]
    
    def matrix_multiply(self, A, B):
        """
        Înmulțește două matrice.
        """
        return [[sum(A[i][k] * B[k][j] for k in range(len(B))) for j in range(len(B[0]))] for i in range(len(A))]
    
    def dot(self, A, B):
        """
        Calculează produsul scalar dintre două vectori.
        """
        return sum(A[i] * B[i] for i in range(len(A)))
    
    def inverse(self, A):
        """
        Calculează inversa unei matrice 2x2.
        """
        determinant = A[0][0] * A[1][1] - A[0][1] * A[1][0]
        if determinant == 0:
            raise ValueError("Matricea nu este inversabilă.")
        else:
            return [[A[1][1] / determinant, -A[0][1] / determinant],
                    [-A[1][0] / determinant, A[0][0] / determinant]]


In [26]:
import pandas as pd
import numpy as np

class AdvancedRegressor:
    def __init__(self):
        self.coefficients = None
    
    def train(self, X, y):
        # Add a column of 1s to X to represent the intercept
        X_with_intercept = self.add_intercept_column(X)

        # Calculate coefficients using the method of least squares
        X_transpose = self.transpose(X_with_intercept)
        XTX_product = self.matrix_multiply(X_transpose, X_with_intercept)
        XTX_inverse = self.gaussian_elimination(XTX_product)

        XTY_product = self.matrix_multiply(X_transpose, y)

        self.coefficients = self.matrix_multiply(XTX_inverse, XTY_product)

        print("Coefficients:", self.coefficients)


    
    def predict(self, X):
        """
        Efectuează predicții pe datele de intrare X.
        """
        # Adăugăm o coloană de 1 la intrările X pentru a reprezenta intercept-ul
        X_with_intercept = self.add_intercept_column(X)

        # Efectuăm predicții folosind coeficienții modelului
        predictions = self.matrix_multiply(X_with_intercept, self.coefficients)
        return predictions
    
    def add_intercept_column(self, X):
        """
        Adaugă o coloană de 1 la matricea de intrare X.
        """
        return [[1] + row for row in X]

    def transpose(self, A):
        """
        Transpune o matrice.
        """
        return [[A[j][i] for j in range(len(A))] for i in range(len(A[0]))]
    
    def matrix_multiply(self, A, B):
        """
        Înmulțește două matrice.
        """
        if len(A[0]) != len(B):
            raise ValueError("Dimensiunile matricelor nu sunt compatibile pentru înmulțire.")
        
        if isinstance(B[0], list):
            # B is a list of lists (matrix)
            result = []
            for i in range(len(A)):
                row = []
                for j in range(len(B[0])):
                    val = sum(A[i][k] * B[k][j] for k in range(len(B)))
                    row.append(val)
                result.append(row)
        else:
            # B is a scalar or NumPy array
            result = [sum(A[i][k] * B[k] for k in range(len(B))) for i in range(len(A))]

        return result

    
    def gaussian_elimination(self, A):
        """
        Metoda de eliminare Gaussiană pentru calculul inversului unei matrice.
        """
        n = len(A)
        inverse = [[0] * n for _ in range(n)]

        # Construim o matrice identitate pentru a stoca inversa
        for i in range(n):
            inverse[i][i] = 1

        # Aplicăm eliminarea Gaussiană
        for i in range(n):
            # Divizăm rândul curent pentru a face elementul diagonală 1
            divisor = A[i][i]
            for j in range(n):
                A[i][j] /= divisor
                inverse[i][j] /= divisor

            # Anulăm toți ceilalți termeni din coloană
            for k in range(n):
                if k != i:
                    factor = A[k][i]
                    for j in range(n):
                        A[k][j] -= factor * A[i][j]
                        inverse[k][j] -= factor * inverse[i][j]
        return inverse


      

def readDatas(dataPath: str):
    df = pd.read_csv(dataPath, delimiter=',', header='infer')
    return df.dropna()

def getTrainingAndValidationSets2(df_world_happiness):
    dataSize = df_world_happiness.shape[0]
    trainingIndexSet = np.random.choice(range(dataSize), size=int(0.8 * dataSize), replace=False)
    validationIndexSet = np.setdiff1d(np.arange(dataSize), trainingIndexSet)

    trainingInputSet = np.array([[df_world_happiness["Economy..GDP.per.Capita."].iloc[index], df_world_happiness["Freedom"].iloc[index]] for index in trainingIndexSet])
    trainingOutputSet = np.array([df_world_happiness["Happiness.Score"].iloc[index] for index in trainingIndexSet])

    validationInputSet = np.array([[df_world_happiness["Economy..GDP.per.Capita."].iloc[index], df_world_happiness["Freedom"].iloc[index]] for index in validationIndexSet])
    validationOutputSet = np.array([df_world_happiness["Happiness.Score"].iloc[index] for index in validationIndexSet])
    
    return trainingInputSet, trainingOutputSet, validationInputSet, validationOutputSet

def happinessByGDPAndFreedom(world_happiness):
    regressor = AdvancedRegressor()
    
    training_input_set, training_output_set, validation_input_set, validation_output_set = getTrainingAndValidationSets2(world_happiness)
    regressor.train(training_input_set, training_output_set)
    
    W = regressor.coefficients
    w0, w1 = W[0], W[1]
    print("f(x) = {} + {}*x1 + {}*x2".format(w0, w1, w2))

    # Eroare
    computed_validation_outputs = regressor.predict(validation_input_set)
    manual_error = sum((t1 - t2) ** 2 for t1, t2 in zip(computed_validation_outputs, validation_output_set)) / len(validation_output_set)
    print("Eroare: {}".format(manual_error))
    print("Shape of coefficients:", len(self.coefficients))
    print("Coefficients:", self.coefficients)


world_happiness_v1 = readDatas("data/v1_world-happiness-report-2017.csv")
happinessByGDPAndFreedom(world_happiness_v1)


Coefficients: [1.8455975608594173, 1.2308691374177982]


NameError: name 'w2' is not defined